In [ ]:
#!/usr/bin/python
#encoding:utf-8
 
import pymysql
import time,datetime
import xlsxwriter
import smtplib
import traceback
from email.mime.multipart import MIMEMultipart   
from email.mime.text import MIMEText
from email.header import Header  
    
 
#zabbix database information:
zdbhost = '192.168.1.xx'
zdbuser = 'zabbix'
zdbpass = 'zabbix'
zdbport = 3306
zdbname = 'zabbix'
 
 # Generated file name:
linuxfilename = 'Linux_Report_Cpu_Mem_%s.xlsx' % time.strftime("%Y-%m-%d", time.localtime())
windowsfilename = 'Windows_Report_Cpu_Mem_%s.xlsx' % time.strftime("%Y-%m-%d", time.localtime())
 
 # Zabbix which packet is generated reports
groupname = 'Linux'
groupname_2 = 'Windows'
 
 # key list need to query [name, table name, key value, value, formatting, data processing divisible]
keys = [
 [ 'CPU idle mean value', 'trends', 'system.cpu.util [, idle]', 'avg', '%. 2f', 1],
 [ 'CPU idle minimum value', 'trends', 'system.cpu.util [, idle]', 'min', '%. 2f', 1],
 [ 'Physical memory size (G)', 'trends_uint', 'vm.memory.size [total]', 'avg', '', 1048576000],
 [ 'Average available memory (in G)', 'trends_uint', 'vm.memory.size [available]', 'avg', '', 1048576000],
 [ 'Swap idle rate%', 'trends', 'system.swap.size [, pfree]', 'avg', '%. 2f', 1],
]
 
keys_2=[
 [ 'Windows CPU usage average', 'trends', r'perf_counter [\\ Processor (_Total) \\% Processor Time]', 'avg', '%. 2f', 1],
 [ 'Windows CPU maximum utilization', 'trends', r'perf_counter [\\ Processor (_Total) \\% Processor Time]', 'max', '%. 2f', 1],
 [ 'Physical memory size (G)', 'trends_uint', 'vm.memory.size [total]', 'avg', '', 1048576000],
 [ 'Average available memory (in G)', 'trends_uint', 'vm.memory.size [free]', 'avg', '', 1048576000],
 [ 'Virtual Memory idle rate%', 'trends', 'vm.vmemory.size [pavailable]', 'avg', '%. 2f', 1],
]
 
class ReportForm:
 
    def __init__(self,groupname):
       
                 # Open Database Connectivity
        self.conn = pymysql.connect(host=zdbhost,user=zdbuser,passwd=zdbpass,port=zdbport,db=zdbname,cursorclass=pymysql.cursors.DictCursor)
        self.cursor = self.conn.cursor()
 
                 # Obtain IP information:
        self.IpInfoList =self.getHostList(groupname)
 
 
         # Obtain the set of all IP group name according to zabbix
    def getgroupid(self,groupname):
 
                 # Query group ID:
 
        sql = '''select groupid from groups where name = '%s' ''' % groupname
 
        self.cursor.execute(sql)
 
        groupid =self.cursor.fetchone()['groupid']
 
        return groupid
 
 
         # The following groupid query packet to all of the host ID (hostid):
    def gethostid(self,groupname):
 
        groupid =self.getgroupid(groupname)
 
        sql = '''select hostid from hosts_groups where groupid = %s''' % groupid
 
        self.cursor.execute(sql)
 
        hostlist = self.cursor.fetchall()
 
        return hostlist
 
 
         # IP information generated dictionary: structure { '119.146.207.19': { 'hostid': 10086L,},}
    def getHostList(self,groupname):
 
        hostlist =self.gethostid(groupname)
 
        IpInfoList = {}
 
        for i in hostlist:
 
            hostid = i['hostid']
 
            sql = '''select host from hosts where status = 0 and available=1 and hostid = %s''' % hostid
 
            ret =self.cursor.execute(sql)
 
            if ret:
 
               IpInfoList[self.cursor.fetchone()['host']] = {'hostid':hostid}
 
        return IpInfoList
 
 
         # Get itemid
    def getItemid(self,hostid,itemname):
 
        sql = '''select itemid from items where hostid = %s and key_ = '%s' ''' % (hostid, itemname)
 
        if self.cursor.execute(sql):
 
            itemid =self.cursor.fetchone()['itemid']
 
        else:
 
            itemid = None
 
        return itemid
 
 
         # Trends table query value, type value of min, max, avg three kinds
    def getTrendsValue(self,type,itemid, start_time, stop_time):
 
        sql = '''select %s(value_%s) as result from trends where itemid = %s and clock >= %s and clock <= %s''' % (type, type, itemid, start_time, stop_time)
 
        self.cursor.execute(sql)
 
        result =self.cursor.fetchone()['result']
 
        if result == None:
 
            result = 0
 
        return result
 
 
         # Trends_uint table query value, type value of min, max, avg three kinds
    def getTrends_uintValue(self,type,itemid, start_time, stop_time):
 
        sql = '''select %s(value_%s) as result from trends_uint where itemid = %s and clock >= %s and clock <= %s''' % (type, type, itemid, start_time, stop_time)
 
        self.cursor.execute(sql)
 
        result =self.cursor.fetchone()['result']
 
        if result:
 
            result = int(result)
 
        else:
 
            result = 0
 
        return result
 
 
 
 
         # According to hostid, itemname get the value of the monitored item
    def getLastWeekData(self,type,hostid,table,itemname):
 
                 # Get the first day and the last day of last week
        self.fromdays=7+datetime.date.today().weekday()
        self.todays=datetime.date.today().weekday()+1
 
        ts_first =int(time.mktime((datetime.date.today()-datetime.timedelta(days=self.fromdays)).timetuple()))
 
        ts_last =int(time.mktime((datetime.date.today()-datetime.timedelta(days=self.todays)).timetuple()))
 
 
 
        itemid =self.getItemid(hostid, itemname)
 
        function =getattr(self,'get%sValue' % table.capitalize())
 
 
 
        return  function(type,itemid, ts_first, ts_last)
 
 
    def getInfo(self):
 
                 # Loop reads the IP list information
 
        for ip,resultdict in  linux.IpInfoList.items():
 
                         print ( "being queried IP:% - 15shostid:% 5d of information!"% (ip, resultdict [ 'hostid']))
 
                 # Loop reads keys, one by one key statistics:
 
        #ip 192.168.10.106    resultdict{'hostid': 10134L}
 
            for value in keys:
 
                 # [ 'CPU idle minimum value', 'trends', 'system.cpu.util [, idle]', 'min', '%. 2f', 1]
 
                                 #print ( "\ t is the statistical key _:% s"% value [2])
 
                if not value[2] in linux.IpInfoList[ip]:
 
                   linux.IpInfoList[ip][value[2]] = {}
 
                data =  linux.getLastWeekData(value[3],resultdict['hostid'],value[1],value[2])
 
                linux.IpInfoList[ip][value[2]][value[3]] = data
 
        #{'192.168.10.213':{'vfs.fs.size[/,total]': {'avg': 52844687360}, 'hostid': 10285L,'vm.memory.size[total]': {'avg': 33615073280}, 'system.swap.size[,total]':{'avg': 16877871104}, 'system.cpu.util[,idle]': {'avg': 97.409428410000004, 'min':94.498400000000004}, 'system.swap.size[,free]': {'avg': 16877871104},'vfs.fs.size[/,free]': {'avg': 45307376183}, 'system.cpu.load[percpu,avg5]':{'avg': 0.00272683}, 'vm.memory.size[available]': {'avg': 33064865249, 'min':32991432704}}
 
    def getInfo2(self):
 
                 # Loop reads the IP list information
 
        for ip,resultdict in  windows.IpInfoList.items():
 
                         print ( "being queried IP:% - 15shostid:% 5d of information!"% (ip, resultdict [ 'hostid']))
 
                 # Loop reads keys, one by one key statistics:
 
        #ip 192.168.10.106    resultdict{'hostid': 10134L}
 
            for value in keys_2:
 
                 # [ 'CPU idle minimum value', 'trends', 'system.cpu.util [, idle]', 'min', '%. 2f', 1]
 
                                 #print ( "\ t is the statistical key _:% s"% value [2])
 
                if not value[2] in windows.IpInfoList[ip]:
 
                   windows.IpInfoList[ip][value[2]] = {}
                
 
                data1 =  windows.getLastWeekData(value[3],resultdict['hostid'],value[1],value[2])
 
                windows.IpInfoList[ip][value[2]][value[3]] = data1
 
        #{'192.168.1.213':{'vfs.fs.size[/,total]': {'avg': 52844687360}, 'hostid': 10285L,'vm.memory.size[total]': {'avg': 33615073280}, 'system.swap.size[,total]':{'avg': 16877871104}, 'system.cpu.util[,idle]': {'avg': 97.409428410000004, 'min':94.498400000000004}, 'system.swap.size[,free]': {'avg': 16877871104},'vfs.fs.size[/,free]': {'avg': 45307376183}, 'system.cpu.load[percpu,avg5]':{'avg': 0.00272683}, 'vm.memory.size[available]': {'avg': 33064865249, 'min':32991432704}}
 
 
         # Generate xls file
    def writeToXls(self,xlsfilename,keysname):
 
                         #Create a file 
            workbook =xlsxwriter.Workbook(xlsfilename)
 
                         # Production time interval
            start_day=datetime.date.today()-datetime.timedelta(days=self.fromdays)
            end_day=datetime.date.today()-datetime.timedelta(days=self.todays)
 
                         # Create a workbook
                         worksheet = workbook.add_worksheet ( '% s to% s'% (start_day, end_day))
                         # Set Column Width
            worksheet.set_column('A:G', 18)
 
                         # Create the Format Cells
            format1 = workbook.add_format()
            format1.set_fg_color('#003063')
            format1.set_font_color('white')
            format1.set_border(1)
 
            format2 = workbook.add_format()
            format2.set_border(1)
 
                         # Write the first column:
            worksheet.write (0,0, "Host Name IP", format1)
            i = 1
            for ip in self.IpInfoList:
               worksheet.write(i,0,ip,format2)
               i = i + 1
 
                         # Write another column (if the number of columns is 5, the write memory percentage):
            i = 1
            k = 1
            for value in keysname:
                if i == 5:
                    k=i+1
                                         worksheet.write (0, i, "percentage of available memory%", format1)
                    worksheet.write(0,k,value[0],format1)
                else:
                    k=i
                    worksheet.write(0,k,value[0],format1)
 
                                 # This column is written content (the percentage of memory divided data obtained by the first two columns):
                j = 1
                for ip,result in self.IpInfoList.items():
                    if i==5:
                       worksheet.write(j,5, '=ROUND(E%s/D%s*100,2)'%(j+1, j+1),format2)
                    if value[4]:
                       worksheet.write(j,k, value[4] % result[value[2]][value[3]],format2)
                    else:
                       worksheet.write(j,k, "{:.2f}".format(result[value[2]][value[3]] / value[5]),format2)
                       
                    j = j + 1
 
                i = i + 1
            
            workbook.close()
 
 
         # Close the database connection
    def __del__(self):
        
 
        self.cursor.close()
 
        self.conn.close()
        
 
 #send email 
def sendmail():
         # Set the required parameters smtplib
         # The following sender, recipient for e-mail transmission
    smtpserver = 'mail.qq.com'
    sender='123@qq.com'
         # Recipients multiple recipients, separated by commas
    to_mail=['123@qq.com']
    cc_mail=['123@qq.com']
        
         # Set Title
         subject = 'test Zabbix monitored weekly: memory, CPU'
    subject=Header(subject, 'utf-8').encode()
        
         # Mail objects MIMEMultipart construction objects
         # The following subject, sender, recipient, date is displayed on the message page.
    msg = MIMEMultipart('mixed') 
    msg['Subject'] = subject
    msg['From'] = '123@qq.com'
         # Recipients multiple recipients and Cc through to join the list into; string of interval
    msg['To'] = ";".join(to_mail)
    msg['Cc'] = ";".join(cc_mail)        
         # Configuration text   
         text = "Zabbix monitoring Weekly: memory, CPU \ n \ n Please check!"    
    text_plain = MIMEText(text,'plain', 'utf-8')    
    msg.attach(text_plain)
        
         # Accessory structure
    linuxfile=open(r'%s' % linuxfilename,'rb').read()
    text_att = MIMEText(linuxfile, 'base64', 'utf-8') 
    text_att["Content-Type"] = 'application/octet-stream'
    text_att["Content-Disposition"] = 'attachment; filename=%s' % linuxfilename
    msg.attach(text_att)
 
    windowsfile=open(r'%s' % windowsfilename,'rb').read()
    text_att2 = MIMEText(windowsfile, 'base64', 'utf-8') 
    text_att2["Content-Type"] = 'application/octet-stream'
    text_att2["Content-Disposition"] = 'attachment; filename=%s' % windowsfilename
    msg.attach(text_att2)
        
         # Send e-mail, sent to the recipient at the same time and Cc
    smtp = smtplib.SMTP()    
    smtp.connect('mail.shanghaitrust.com', '25')
    smtp.sendmail(sender, to_mail+cc_mail, msg.as_string())  
    smtp.quit()
 
 
if __name__ == "__main__":
    try:
        
        linux = ReportForm(groupname)
 
        linux.getInfo()
 
        linux.writeToXls(linuxfilename,keys)
 
        windows = ReportForm(groupname_2)
 
        windows.getInfo2()
 
        windows.writeToXls(windowsfilename,keys_2)
 
        sendmail()
    except:
                 # Catch the exception, written to the log
        f=open("zabbix_log.txt",'a')  
        traceback.print_exc(file=f)  
        f.flush()  
        f.close()   
